In [1]:
#Import Packages
import pandas as pd
import numpy as np

In [6]:
#read in data
month1 = pd.read_csv('data/train_month_1.csv')
month2 = pd.read_csv('data/train_month_2.csv')
month3 = pd.read_csv('data/train_month_3_with_target.csv')

Between the first and second month data, we try to figure out if certain columns changes. We then store the booleans (True/False) in a python list to indicate which features change and which remain constant. Note that we set client_id to True, so we can merge the final datasets together using this column

In [16]:
notequal = []
for col in list(month1):
    notequal.append(not month1[col].equals(month2[col]))
notequal[0] = True
print(notequal)

[True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, True, False, True, True]


Repeat for the first and third month. Note that we  append an extra True value at the end, because the third month dataset has an additional column (Target), that was previously not available in the other two datasets.

In [17]:
notequal2 = []
for col in list(month1):
    notequal2.append(not month1[col].equals(month3[col]))
notequal2.append(True)
notequal2[0] = True

print(notequal2)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, True, False, True, True, True]


In [73]:
month1.columns

Index(['client_id', 'homebanking_active', 'has_homebanking',
       'has_insurance_21', 'has_insurance_23', 'has_life_insurance_fixed_cap',
       'has_life_insurance_decreasing_cap', 'has_fire_car_other_insurance',
       'has_personal_loan', 'has_mortgage_loan', 'has_current_account',
       'has_pension_saving', 'has_savings_account',
       'has_savings_account_starter', 'has_current_account_starter',
       'bal_insurance_21', 'bal_insurance_23', 'cap_life_insurance_fixed_cap',
       'cap_life_insurance_decreasing_cap', 'prem_fire_car_other_insurance',
       'bal_personal_loan', 'bal_mortgage_loan', 'bal_current_account',
       'bal_pension_saving', 'bal_savings_account',
       'bal_savings_account_starter', 'bal_current_account_starter',
       'visits_distinct_so', 'visits_distinct_so_areas', 'customer_since_all',
       'customer_since_bank', 'customer_gender', 'customer_birth_date',
       'customer_postal_code', 'customer_occupation_code',
       'customer_self_employed',

In [18]:
#rename the columns in month 3, to prevent features from replacing each other when merging
month3 = month3.add_suffix("_3")
month3 = month3.rename(columns={"client_id_3":"client_id", "target_3":"target"})
month3.head()

,client_id,homebanking_active_3,has_homebanking_3,has_insurance_21_3,has_insurance_23_3,has_life_insurance_fixed_cap_3,has_life_insurance_decreasing_cap_3,has_fire_car_other_insurance_3,has_personal_loan_3,has_mortgage_loan_3,...,customer_since_bank_3,customer_gender_3,customer_birth_date_3,customer_postal_code_3,customer_occupation_code_3,customer_self_employed_3,customer_education_3,customer_children_3,customer_relationship_3,target
0,910df42ad36243aa4ce16324cd7b15b0,0,0,0,0,0,0,1,0,0,...,1994-08,1,1943-09,3630,9.0,0,0.0,NaN,NaN,0
1,4e19dc3a54323c5bbfc374664b950cd1,1,1,0,0,0,0,0,0,0,...,2017-01,1,1994-02,2460,9.0,0,NaN,mature,couple,0
2,f5d08db1b86c0cb0f566bf446cff1fb4,1,1,0,0,0,0,1,0,0,...,1980-12,2,1936-10,2660,9.0,0,NaN,NaN,single,0
3,26170ecf63653e215c52f4262c1c4859,0,0,0,0,0,0,1,0,0,...,2013-10,1,1946-09,6600,9.0,0,NaN,NaN,NaN,0
4,c078009957dffb64f20e61b41220a976,0,0,0,0,0,0,0,0,0,...,2012-11,2,1996-04,8550,9.0,0,NaN,mature,couple,1


Next, we merge the three datasets, only joining features that change from month to months. Anything constant such as date of birth only appear once in the final dataset. All features that appear repeatedly from month to month will have a suffix of a number indicating the feature for the particular month. For example, "has_homebanking_1" is the associated feature that indicates whether a client has homebanking for month 1, while "has_homebanking_2" is for month 2, etc.

In [19]:
full_df = pd.merge(month1, month2.loc[:,notequal], left_on="client_id", right_on="client_id", how="inner", suffixes=["_1", "_2"])
full_df = pd.merge(full_df, month3.loc[:,notequal2], left_on="client_id", right_on="client_id", how="inner")
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,bal_pension_saving_3,bal_savings_account_3,bal_savings_account_starter_3,bal_current_account_starter_3,visits_distinct_so_3,visits_distinct_so_areas_3,customer_self_employed_3,customer_children_3,customer_relationship_3,target
0,910df42ad36243aa4ce16324cd7b15b0,0,0,0,0,0,0,1,0,0,...,0,22000,0,0,1.0,1.0,0,NaN,NaN,0
1,4e19dc3a54323c5bbfc374664b950cd1,1,1,0,0,0,0,0,0,0,...,0,10570,0,0,1.0,1.0,0,mature,couple,0
2,f5d08db1b86c0cb0f566bf446cff1fb4,1,1,0,0,0,0,1,0,0,...,0,15200,0,0,1.0,1.0,0,NaN,single,0
3,26170ecf63653e215c52f4262c1c4859,0,0,0,0,0,0,1,0,0,...,0,29020,0,0,1.0,1.0,0,NaN,NaN,0
4,c078009957dffb64f20e61b41220a976,0,0,0,0,0,0,0,0,0,...,0,13650,0,0,1.0,1.0,0,mature,couple,1


To avoid having multiple repeated features for different months, we combine them either in sequences of binary or an aggregate of count.

In [20]:
#these features have a binary value indicating presence/absence
status_features = list(month1.loc[:, notequal].columns)[:14]
status_features

['client_id',
 'homebanking_active',
 'has_homebanking',
 'has_insurance_21',
 'has_insurance_23',
 'has_life_insurance_fixed_cap',
 'has_life_insurance_decreasing_cap',
 'has_fire_car_other_insurance',
 'has_personal_loan',
 'has_mortgage_loan',
 'has_current_account',
 'has_pension_saving',
 'has_savings_account',
 'has_current_account_starter']

For the first 13 features (excluding client_id), we can concat the binary features into a string sequence to indicate a 3 month combination pattern:

In [21]:
for col in status_features:
    if col != "client_id":
        
        full_df[str(col) + "_seq"] = full_df.filter(regex="^"+str(col)+"_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

In [22]:
full_df.filter(regex=".*_seq").head()

,homebanking_active_seq,has_homebanking_seq,has_insurance_21_seq,has_insurance_23_seq,has_life_insurance_fixed_cap_seq,has_life_insurance_decreasing_cap_seq,has_fire_car_other_insurance_seq,has_personal_loan_seq,has_mortgage_loan_seq,has_current_account_seq,has_pension_saving_seq,has_savings_account_seq,has_current_account_starter_seq
0,000,000,000,000,000,000,111,000,000,111,000,111,000
1,111,111,000,000,000,000,000,000,000,111,000,111,000
2,111,111,000,000,000,000,111,000,000,111,000,111,000
3,000,000,000,000,000,000,111,000,000,000,000,111,000
4,000,000,000,000,000,000,000,000,000,000,000,111,000


We can also simply aggregate the binary values, to count how many positive occurances a particular client had in the 3 month span for these 14 features:

In [23]:
for col in status_features:
    if col != "client_id":
        
        full_df[str(col) + "_count"] = full_df.filter(regex="^"+str(col)+"_[0-9]").sum(axis=1)

In [35]:
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,has_insurance_23_count,has_life_insurance_fixed_cap_count,has_life_insurance_decreasing_cap_count,has_fire_car_other_insurance_count,has_personal_loan_count,has_mortgage_loan_count,has_current_account_count,has_pension_saving_count,has_savings_account_count,has_current_account_starter_count
0,910df42ad36243aa4ce16324cd7b15b0,0,0,0,0,0,0,1,0,0,...,0,0,0,3,0,0,3,0,3,0
1,4e19dc3a54323c5bbfc374664b950cd1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,3,0
2,f5d08db1b86c0cb0f566bf446cff1fb4,1,1,0,0,0,0,1,0,0,...,0,0,0,3,0,0,3,0,3,0
3,26170ecf63653e215c52f4262c1c4859,0,0,0,0,0,0,1,0,0,...,0,0,0,3,0,0,0,0,3,0
4,c078009957dffb64f20e61b41220a976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0


For Balance features, we take the difference between the third and first month, to track the change over time. Note that this also includes visits

In [24]:
balance_features = list(month1.loc[:, notequal].columns)[14:28]
balance_features

['bal_insurance_21',
 'bal_insurance_23',
 'cap_life_insurance_fixed_cap',
 'cap_life_insurance_decreasing_cap',
 'prem_fire_car_other_insurance',
 'bal_personal_loan',
 'bal_mortgage_loan',
 'bal_current_account',
 'bal_pension_saving',
 'bal_savings_account',
 'bal_savings_account_starter',
 'bal_current_account_starter',
 'visits_distinct_so',
 'visits_distinct_so_areas']

In [27]:
for col in balance_features:
    full_df[str(col)+"_diff"] = np.array(full_df.filter(regex="^"+str(col)+"_3")) - np.array(full_df.filter(regex="^"+str(col)+"_1"))

Convert Column into datetime object

In [28]:
full_df['customer_since_all_d']=pd.to_datetime(full_df['customer_since_all'], format='%Y-%m-%d')
full_df['customer_birth_date_d']=pd.to_datetime(full_df['customer_birth_date'], format='%Y-%m-%d')

Compute the age of the client

In [29]:
full_df['age']=2018-pd.DatetimeIndex(full_df['customer_birth_date_d']).year
full_df['mob']=((pd.to_datetime('2018-10-01')-full_df['customer_birth_date_d'])/np.timedelta64(1, 'M')).astype('int')

Instead of looking at each feature's sequence or count separately e.g `has_homebanking_seq`, we make a column `insurance` is a higher level feature that is 1 if either has_insurance_21, has_insurance_23, has_life_insurance_fixed_cap, etc. is 1 otherwise it is 0. We do this for all three months and at the end we can have `insurance_seq`, which is the 3 month sequence of the higher level variable.

In [30]:
full_df.columns

Index(['client_id', 'homebanking_active_1', 'has_homebanking_1',
       'has_insurance_21_1', 'has_insurance_23_1',
       'has_life_insurance_fixed_cap_1', 'has_life_insurance_decreasing_cap_1',
       'has_fire_car_other_insurance_1', 'has_personal_loan_1',
       'has_mortgage_loan_1',
       ...
       'bal_pension_saving_diff', 'bal_savings_account_diff',
       'bal_savings_account_starter_diff', 'bal_current_account_starter_diff',
       'visits_distinct_so_diff', 'visits_distinct_so_areas_diff',
       'customer_since_all_d', 'customer_birth_date_d', 'age', 'mob'],
      dtype='object', length=145)

In [36]:
full_df['insurance_3']=full_df[['has_insurance_21_3','has_insurance_23_3','has_life_insurance_fixed_cap_3','has_life_insurance_decreasing_cap_3','has_fire_car_other_insurance_3']].apply(lambda row: row.max(),axis=1)
full_df['insurance_2']=full_df[['has_insurance_21_2','has_insurance_23_2','has_life_insurance_fixed_cap_2','has_life_insurance_decreasing_cap_2','has_fire_car_other_insurance_2']].apply(lambda row: row.max(),axis=1)
full_df['insurance_1']=full_df[['has_insurance_21_1','has_insurance_23_1','has_life_insurance_fixed_cap_1','has_life_insurance_decreasing_cap_1','has_fire_car_other_insurance_1']].apply(lambda row: row.max(),axis=1)
full_df['savings_3']=full_df[['has_pension_saving_3','has_savings_account_3','has_savings_account_starter_3','has_current_account_starter_3']].apply(lambda row: row.max(),axis=1)
#note has_savings_account_starter did not change from month 1 to month 2 for all clients
full_df['savings_2']=full_df[['has_pension_saving_2','has_savings_account_2','has_savings_account_starter','has_current_account_starter_2']].apply(lambda row: row.max(),axis=1)
full_df['savings_1']=full_df[['has_pension_saving_1','has_savings_account_1','has_savings_account_starter','has_current_account_starter_1']].apply(lambda row: row.max(),axis=1)
full_df['loan_3']=full_df[['has_personal_loan_3','has_mortgage_loan_3','has_current_account_3']].apply(lambda row: row.max(),axis=1)
full_df['loan_2']=full_df[['has_personal_loan_2','has_mortgage_loan_2','has_current_account_2']].apply(lambda row: row.max(),axis=1)
full_df['loan_1']=full_df[['has_personal_loan_1','has_mortgage_loan_1','has_current_account_1']].apply(lambda row: row.max(),axis=1)

#checks whether the customer has an account with the bank for a particular month
full_df['has_account_3']=full_df[['insurance_3','savings_3','loan_3']].apply(lambda row: row.max(),axis=1)
full_df['has_account_2']=full_df[['insurance_2','savings_2','loan_2']].apply(lambda row: row.max(),axis=1)
full_df['has_account_1']=full_df[['insurance_1','savings_1','loan_1']].apply(lambda row: row.max(),axis=1)


#Total balance for a particular month
full_df['bal_insurance_3']=full_df[['bal_insurance_21_3','bal_insurance_23_3']].apply(lambda row: row.sum(),axis=1)
full_df['bal_insurance_2']=full_df[['bal_insurance_21_2','bal_insurance_23_2']].apply(lambda row: row.sum(),axis=1)
full_df['bal_insurance_1']=full_df[['bal_insurance_21_1','bal_insurance_23_1']].apply(lambda row: row.sum(),axis=1)

full_df['bal_savings_3']=full_df[['bal_pension_saving_3','bal_savings_account_3','bal_savings_account_starter_3','bal_current_account_starter_3']].apply(lambda row: row.sum(),axis=1)
full_df['bal_savings_2']=full_df[['bal_pension_saving_2','bal_savings_account_2','bal_savings_account_starter_2','bal_current_account_starter_2']].apply(lambda row: row.sum(),axis=1)
full_df['bal_savings_1']=full_df[['bal_pension_saving_1','bal_savings_account_1','bal_savings_account_starter_1','bal_current_account_starter_1']].apply(lambda row: row.sum(),axis=1)

full_df['bal_loan_3']=full_df[['bal_personal_loan_3','bal_mortgage_loan_3','bal_current_account_3']].apply(lambda row: row.sum(),axis=1)
full_df['bal_loan_2']=full_df[['bal_personal_loan_2','bal_mortgage_loan_2','bal_current_account_2']].apply(lambda row: row.sum(),axis=1)
full_df['bal_loan_1']=full_df[['bal_personal_loan_1','bal_mortgage_loan_1','bal_current_account_1']].apply(lambda row: row.sum(),axis=1)

#debt ratio for each month
full_df['debt_ratio_3']=np.where(full_df['bal_loan_3']/full_df['bal_savings_3']>=0,full_df['bal_loan_3']/full_df['bal_savings_3'],0)
full_df['debt_ratio_2']=np.where(full_df['bal_loan_2']/full_df['bal_savings_2']>=0,full_df['bal_loan_2']/full_df['bal_savings_2'],0)
full_df['debt_ratio_1']=np.where(full_df['bal_loan_1']/full_df['bal_savings_1']>=0,full_df['bal_loan_1']/full_df['bal_savings_1'],0)

#gender
full_df['gender'] = np.where(full_df['customer_gender']==1, 1, 0)

In [53]:
#sumarize the binary variables into sequences rather than keeping individual columns

sequence_var = ['insurance', 'savings', 'loan', 'has_account']

for col in sequence_var:
    full_df[str(col)+"_seq"] = full_df.filter(regex="^" +str(col) +"_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

#count gives the number of months the indicator was present in a 3-month span
for col in sequence_var:
    full_df[str(col)+"_count"] = full_df.filter(regex="^" +str(col) +"_[0-9]").sum(axis=1)


In [54]:
#for balances, we take the difference between the third month and the first month

balance_var = ['bal_insurance', 'bal_savings', 'bal_loan']

for col in balance_var:
    full_df[str(col)+"_diff"] = np.array(full_df.filter(regex="^" + str(col)+"_3")) - np.array(full_df.filter(regex="^" + str(col)+"_1"))

In [74]:
#customer self employed status summarized

full_df['customer_self_employed_seq'] = full_df.filter(regex="^customer_self_employed_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

full_df['customer_self_employed_count'] = full_df.filter(regex="^customer_self_employed_[0-9]").sum(axis=1)





#customer relationship

# Customer Children

The current possible values are: NA, yes, no, adolescent, young, baby, mature, etc. We will remap these values to 0, 1, 2. Here 0 indicates missing, 1 indicates no and 2 indicates a children present (regardless of age). We will then proceed with creating sequence variable

In [86]:
def clean_children(text):
    if pd.isnull(text):
        new_val = 0
    elif str(text) == "no":
        new_val = 1
    else:
        new_val = 2
    
    return new_val

full_df["customer_children_clean_1"]  = full_df['customer_children_1'].apply(clean_children)
full_df["customer_children_clean_2"]  = full_df['customer_children_2'].apply(clean_children)
full_df["customer_children_clean_3"]  = full_df['customer_children_3'].apply(clean_children)

full_df['customer_children_seq'] = full_df.filter(regex="^customer_children_clean_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

full_df['customer_children_seq'].value_counts()


000    22380
111    22115
222    17103
220      611
001      497
110      366
221      267
002      174
112      165
101        5
200        5
202        2
100        2
011        2
022        2
012        1
Name: customer_children_seq, dtype: int64

# Customer Relationship

Customer relationship can take the following values: NA, single, couple. We will remap it as 0, 1, 2 respectively and compute the sequence of the relationship over a 3 month period

In [87]:
def clean_relationship(status):
    if pd.isnull(status):
        new_val = 0
    elif status == "single":
        new_val = 1
    else:
        new_val = 2

    return new_val

full_df["customer_relationship_clean_1"]  = full_df['customer_relationship_1'].apply(clean_children)
full_df["customer_relationship_clean_2"]  = full_df['customer_relationship_2'].apply(clean_children)
full_df["customer_relationship_clean_3"]  = full_df['customer_relationship_3'].apply(clean_children)

full_df['customer_relationship_seq'] = full_df.filter(regex="^customer_relationship_clean_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

full_df['customer_relationship_seq'].value_counts()

222    48220
000    13884
220     1000
002      571
200       15
202        6
022        1
Name: customer_relationship_seq, dtype: int64

In [88]:
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,bal_loan_diff,customer_self_employed_seq,customer_children_clean_1,customer_children_clean_2,customer_children_clean_3,customer_children_seq,customer_relationship_clean_1,customer_relationship_clean_2,customer_relationship_clean_3,customer_relationship_seq
0,910df42ad36243aa4ce16324cd7b15b0,0,0,0,0,0,0,1,0,0,...,-370,000,0,0,0,000,0,0,0,000
1,4e19dc3a54323c5bbfc374664b950cd1,1,1,0,0,0,0,0,0,0,...,560,000,2,2,2,222,2,2,2,222
2,f5d08db1b86c0cb0f566bf446cff1fb4,1,1,0,0,0,0,1,0,0,...,420,000,0,0,0,000,2,2,2,222
3,26170ecf63653e215c52f4262c1c4859,0,0,0,0,0,0,1,0,0,...,0,000,0,0,0,000,0,0,0,000
4,c078009957dffb64f20e61b41220a976,0,0,0,0,0,0,0,0,0,...,0,000,2,2,2,222,2,2,2,222


In [90]:
list(full_df.filter(regex="^(.*_[0-9]$)"))

False

In [113]:
#exclude all features with _[0-9]

exclude_lst = list(full_df.filter(regex="^(.*_[0-9]$)"))

exclude_lst.remove("debt_ratio_3")
exclude_lst.remove("debt_ratio_2")
exclude_lst.remove("debt_ratio_1")

final_df = full_df.drop(exclude_lst, axis=1)

In [117]:
final_df.head()

,client_id,has_savings_account_starter,customer_since_all,customer_since_bank,customer_gender,customer_birth_date,customer_postal_code,customer_occupation_code,customer_education,target,...,insurance_count,savings_count,loan_count,has_account_count,bal_insurance_diff,bal_savings_diff,bal_loan_diff,customer_self_employed_seq,customer_children_seq,customer_relationship_seq
0,910df42ad36243aa4ce16324cd7b15b0,0,1983-03,1994-08,1,1943-09,3630,9.0,0.0,0,...,3,3,3,3,0,2000,-370,000,000,000
1,4e19dc3a54323c5bbfc374664b950cd1,0,2017-01,2017-01,1,1994-02,2460,9.0,NaN,0,...,0,3,3,3,0,280,560,000,222,222
2,f5d08db1b86c0cb0f566bf446cff1fb4,0,1980-12,1980-12,2,1936-10,2660,9.0,NaN,0,...,3,3,3,3,0,-1440,420,000,000,222
3,26170ecf63653e215c52f4262c1c4859,0,1998-08,2013-10,1,1946-09,6600,9.0,NaN,0,...,3,3,0,3,0,0,0,000,000,000
4,c078009957dffb64f20e61b41220a976,0,2012-11,2012-11,2,1996-04,8550,9.0,NaN,1,...,0,3,0,3,0,200,0,000,222,222


In [118]:
final_df.to_csv("data_train_final.csv", index=False)